# Pattern Recognition Example

This notebook demonstrates how to use Time Series RAG for pattern recognition tasks.

**Note**: This is a documentation example. To run this notebook, make sure you have installed the package and its dependencies:

```bash
pip install -e .
```

# Pattern Recognition Example

This notebook demonstrates how to use Time Series RAG for pattern recognition tasks.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from timeseries_rag.models import TimeSeriesEmbedder
from timeseries_rag.rag import TimeSeriesRAG, TimeSeriesDocument
from timeseries_rag.analytics import TimeSeriesAnalytics

## Load Example Data

In [ ]:
# Load synthetic patterns
sine_df = pd.read_csv('../../../data/synthetic/sine_amplitude.csv')
composite_df = pd.read_csv('../../../data/synthetic/composite.csv')
square_df = pd.read_csv('../../../data/synthetic/square_noisy.csv')
trend_df = pd.read_csv('../../../data/synthetic/trend_seasonal.csv')

## Extract and Analyze Patterns

In [ ]:
# Initialize analytics
analytics = TimeSeriesAnalytics(sine_df['value'].values)

# Extract common patterns
patterns = analytics.extract_patterns(window_size=24, n_patterns=3)

# Plot patterns
plt.figure(figsize=(15, 5))
for i, pattern in enumerate(patterns):
    plt.subplot(1, 3, i+1)
    plt.plot(pattern['pattern'])
    plt.title(f'Pattern {i+1}\nFrequency: {pattern["frequency"]:.2f}')
plt.tight_layout()
plt.show()

## Search for Similar Patterns

In [ ]:
# Initialize RAG system
embedder = TimeSeriesEmbedder()
rag = TimeSeriesRAG()

# Add patterns to database
for i, pattern in enumerate(patterns):
    embedding = embedder.embed(pattern['pattern'])
    doc = TimeSeriesDocument(
        id=f'pattern_{i}',
        data=pattern['pattern'],
        metadata={'frequency': pattern['frequency']},
        embedding=embedding
    )
    rag.add_document(doc)

# Search for similar patterns
query = patterns[0]['pattern']
query_embedding = embedder.embed(query)
results = rag.search(query_embedding, k=2)

# Plot results
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(query)
plt.title('Query Pattern')

plt.subplot(1, 2, 2)
for result in results:
    plt.plot(result['data'], label=f'Distance: {result["distance"]:.2f}')
plt.title('Similar Patterns')
plt.legend()
plt.tight_layout()
plt.show()